<a href="https://colab.research.google.com/github/KorStats/skku/blob/main/(%EC%B5%9C%EC%A2%852)_kobert_nsmc2_original.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ÌïÑÏöî ÎùºÏù¥Î∏åÎü¨Î¶¨ ÏÑ§Ïπò

In [29]:
!pip install transformers datasets accelerate scikit-learn optuna sentencepiece

In [30]:
!pip install transformers

In [31]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, classification_report
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import torch
import optuna
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import re
import platform
import transformers
import joblib

In [32]:
# GPU ÏÇ¨Ïö© Ïó¨Î∂Ä ÌôïÏù∏
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


# 80% Îç∞Ïù¥ÌÑ∞ÏÖã

In [33]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
'Colab Notebooks'  'My Drive'   ·Ñã·Öµ·Ñå·Ö•·Ü´   ·Ñå·Ö©·ÜØ·Ñã·Ö•·Ü∏·ÑÇ·Ö©·Ü´·ÑÜ·ÖÆ·Ü´


In [34]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/·Ñå·Ö©·ÜØ·Ñã·Ö•·Ü∏·ÑÇ·Ö©·Ü´·ÑÜ·ÖÆ·Ü´/NSMC2_train_·Ñá·ÖÆ·ÜØ·ÑÄ·Ö≤·Ü´·Ñí·Öß·Üº_0.8.csv', encoding='utf-8-sig')

In [35]:
df['label'].value_counts()

label
1    5000
0    4000
Name: count, dtype: int64

In [36]:
df_test=pd.read_excel('/content/gdrive/MyDrive/·Ñå·Ö©·ÜØ·Ñã·Ö•·Ü∏·ÑÇ·Ö©·Ü´·ÑÜ·ÖÆ·Ü´/DATA/NSMC_test.xlsx')

In [37]:
data_list = []
for q, label in zip(df['document'], df['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

#from sklearn.model_selection import train_test_split
#dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0, shuffle=True)

train=pd.DataFrame(data_list)

data_list = []
for q, label in zip(df_test['document'], df_test['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

test=pd.DataFrame(data_list)

train.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)
test.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)

In [38]:
def clean_korean_documents(documents):
    # ÌÖçÏä§Ìä∏ Ï†ïÏ†ú (ÌäπÏàòÍ∏∞Ìò∏Îßå Ï†úÍ±∞)
    for i, document in tqdm(enumerate(documents), total=len(documents), desc="Cleaning Special Characters"):
        document = re.sub(r'[^\w\s„Ñ±-„Ö£Í∞Ä-Ìû£]', '', document)  # ÌäπÏàòÍ∏∞Ìò∏Îßå Ï†úÍ±∞, Ï†ïÍ∑ú ÌëúÌòÑÏãù
        documents[i] = document
    return documents

def remove_words_with_numbers(text):
    return ' '.join([word for word in text.split(',') if not re.search(r'\d', word)])

# ÌïúÍ∏ÄÏù¥ ÏïÑÎãå Î¨∏ÏûêÎ•º Ï†úÍ±∞ÌïòÎäî Ìï®Ïàò
def remove_non_korean(text):
    return re.sub(r'[^„Ñ±-„Ö£Í∞Ä-Ìû£\s]', '', text)

In [39]:
train['text'] = clean_korean_documents(train['text'])
train['text'] = train['text'].apply(lambda x: ','.join(x.split()))
train['text'] = train['text'].apply(remove_words_with_numbers)
train['text'] = train['text'].apply(remove_non_korean)

test['text'] = clean_korean_documents(test['text'])
test['text'] = test['text'].apply(lambda x: ','.join(x.split()))
test['text'] = test['text'].apply(remove_words_with_numbers)
test['text'] = test['text'].apply(remove_non_korean)


Cleaning Special Characters: 100%|‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà| 2000/2000 [00:00<00:00, 12318.07it/s]


In [40]:
# 3. ÌïôÏäµ Îç∞Ïù¥ÌÑ∞ÏôÄ ÌÖåÏä§Ìä∏ Îç∞Ïù¥ÌÑ∞ÏóêÏÑú ÌïÑÏöîÌïú Ïó¥ Ï∂îÏ∂ú Î∞è ÎùºÎ≤® Ïù∏ÏΩîÎî©
X_train = train['text']
y_train = train['category']
X_test = test['text']
y_test=test['category']

# 4. ÎùºÎ≤® Ïù∏ÏΩîÎî©
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded=label_encoder.fit_transform(y_test)
num_labels = len(label_encoder.classes_)

In [41]:
# 5. KoBERT Î™®Îç∏ÏùÑ ÏúÑÌïú ÌÜ†ÌÅ¨ÎÇòÏù¥Ï†Ä Î∞è Îç∞Ïù¥ÌÑ∞ÏÖã ÏÉùÏÑ±
model_name = 'monologg/kobert'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [42]:
# ÌÖçÏä§Ìä∏Î•º ÌÜ†ÌÅ∞ÌôîÌïòÎäî Ìï®Ïàò Ï†ïÏùò
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=256)

In [43]:
# Îç∞Ïù¥ÌÑ∞ÏÖã ÏÉùÏÑ± Î∞è ÌÜ†ÌÅ¨ÎÇòÏù¥Ï¶à
train_df = pd.DataFrame({'text': X_train, 'label': y_train_encoded})
test_df = pd.DataFrame({'text': X_test, 'label': y_test_encoded})
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [44]:
# PyTorch ÌÖêÏÑú ÌòïÏãùÏúºÎ°ú Î≥ÄÌôò
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# 5. KoBERT Î™®Îç∏ ÏÑ§Ï†ï Î∞è ÌïôÏäµ ÌååÎùºÎØ∏ÌÑ∞ ÏµúÏ†ÅÌôî
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [45]:
#CPU Ï∏°Ï†ï

import psutil
import time
from transformers import Trainer, TrainingArguments

# CPU ÏÇ¨Ïö©Îüâ Î∞è ÏÜçÎèÑ Ï∏°Ï†ï Ìï®Ïàò
def measure_cpu_usage_and_time(train_function):
    start_time = time.time()  # ÏãúÏûë ÏãúÍ∞Ñ Í∏∞Î°ù
    start_cpu = psutil.cpu_percent(interval=None)  # ÏãúÏûë CPU ÏÇ¨Ïö©Îüâ

    train_function()  # ÌïôÏäµ Ìï®Ïàò Ìò∏Ï∂ú

    end_time = time.time()  # Ï¢ÖÎ£å ÏãúÍ∞Ñ Í∏∞Î°ù
    end_cpu = psutil.cpu_percent(interval=None)  # Ï¢ÖÎ£å CPU ÏÇ¨Ïö©Îüâ

    elapsed_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    print(f"Elapsed Time: {elapsed_time:.2f} seconds")
    print(f"Average CPU Usage: {avg_cpu_usage:.2f}%")

In [46]:
import optuna
from transformers import Trainer, TrainingArguments
from sklearn.metrics import f1_score
import numpy as np

# `compute_metrics` Ìï®Ïàò
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='macro')  # F1 Ï†êÏàò Í≥ÑÏÇ∞
    return {'eval_f1': f1}  # 'eval_f1' Î∞òÌôò

# OptunaÎ•º ÌôúÏö©Ìïú ÌïòÏù¥ÌçºÌååÎùºÎØ∏ÌÑ∞ ÌäúÎãù
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

In [47]:
training_args = TrainingArguments(
    output_dir='/content',
    num_train_epochs=10,  # ÏóêÌè¨ÌÅ¨ Ïàò Í≥†Ï†ï
    per_device_train_batch_size=64,
    learning_rate=5.0e-05,  # ÏµúÏ†ÅÌôîÎêú ÌïôÏäµÎ•†
    weight_decay=0.01,  # ÏµúÏ†ÅÌôîÎêú weight decay
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='/content',
    logging_steps=10,
    fp16=True,
    optim="adamw_torch",  # AdamW fused Ï†ÅÏö© (PyTorch 2.0 Ïù¥ÏÉÅ)
    disable_tqdm=False,
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # Í∞ÄÏû• Ï¢ãÏùÄ Î™®Îç∏ÏùÑ accuracy Í∏∞Ï§ÄÏúºÎ°ú ÏÑ†ÌÉù
    greater_is_better=True,  # accuracyÍ∞Ä ÎÜíÏùÄ Î™®Îç∏ÏùÑ Ï†ÄÏû•
    save_total_limit=2,  # Ï†ÄÏû•Ìï† Î™®Îç∏ Í∞úÏàò Ï†úÌïú
    report_to="none"
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of ü§ó Transformers. Use `eval_strategy` instead
  warnings.warn(


In [48]:
# ÏµúÏ†ÅÏùò ÌååÎùºÎØ∏ÌÑ∞Î°ú Trainer Íµ¨ÏÑ±
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

trainer7 = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,  # Ï†ÑÏ≤¥ ÌïôÏäµ Îç∞Ïù¥ÌÑ∞Î°ú ÌèâÍ∞Ä
    compute_metrics=compute_metrics
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
# CPU ÏÇ¨Ïö©Îüâ Î∞è ÏÜçÎèÑ Ï∏°Ï†ï
measure_cpu_usage_and_time(trainer7.train)

Elapsed Time: 1006.22 seconds
Average CPU Usage: 10.05%


In [50]:
# Î™®Îç∏ Ï†ÄÏû•
joblib.dump(trainer7, '/content/gdrive/MyDrive/NSMC2_0.8_original.pkl')

print("Î™®Îç∏Ïù¥ Ï†ÄÏû•ÎêòÏóàÏäµÎãàÎã§.")

Î™®Îç∏Ïù¥ Ï†ÄÏû•ÎêòÏóàÏäµÎãàÎã§.


In [51]:
# 8. Ï†ÄÏû•Îêú Î™®Îç∏Î°ú ÌÖåÏä§Ìä∏ Îç∞Ïù¥ÌÑ∞ ÏòàÏ∏° Î∞è Í≤∞Í≥º Ï†ÄÏû•
# Ï†ÄÏû•Îêú Î™®Îç∏ Î∂àÎü¨Ïò§Í∏∞

loaded_model = joblib.load('/content/gdrive/MyDrive/NSMC2_0.8_original.pkl')

# Î∂àÎü¨Ïò® Î™®Îç∏ÏùÑ ÏÇ¨Ïö©ÌïòÏó¨ ÏòàÏ∏° ÏàòÌñâ
predictions7 = loaded_model.predict(test_dataset)

test_predictions7 = np.argmax(predictions7.predictions, axis=1)
decoded_test_predictions7 = label_encoder.inverse_transform(test_predictions7)

In [52]:
test["category_prediction"] = decoded_test_predictions7

selected_columns7 = test[['text', 'category_prediction']]
selected_columns7.head(10)

text category_prediction
0                                Ïñ¥Î¶¥ÎïåÎ≥¥Í≥† ÏßÄÍ∏àÎã§ÏãúÎ¥êÎèÑ Ïû¨Î∞åÏñ¥Ïöî„Öã„Öã                   1
1  ÎîîÏûêÏù∏ÏùÑ Î∞∞Ïö∞Îäî ÌïôÏÉùÏúºÎ°ú Ïô∏Íµ≠ÎîîÏûêÏù¥ÎÑàÏôÄ Í∑∏Îì§Ïù¥ ÏùºÍµ∞ Ï†ÑÌÜµÏùÑ ÌÜµÌï¥ Î∞úÏ†ÑÌï¥Í∞ÄÎäî Î¨∏ÌôîÏÇ∞ÏóÖ...                   1
2                      Ìè¥Î¶¨Ïä§Ïä§ÌÜ†Î¶¨ ÏãúÎ¶¨Ï¶àÎäî Îâ¥ÍπåÏßÄ Î≤ÑÎ¶¥Íªò ÌïòÎÇòÎèÑ ÏóÜÏùå ÏµúÍ≥†                   0
3   ÏôÄ Ïó∞Í∏∞Í∞Ä ÏßÑÏßú Í∞úÏ©îÍµ¨ÎÇò ÏßÄÎ£®Ìï†Í±∞ÎùºÍ≥† ÏÉùÍ∞ÅÌñàÎäîÎç∞ Î™∞ÏûÖÌï¥ÏÑú Î¥§Îã§ Í∑∏Îûò Ïù¥Îü∞Í≤å ÏßÑÏßú ÏòÅÌôîÏßÄ                   1
4                         ÏïàÍ∞ú ÏûêÏö±Ìïú Î∞§ÌïòÎäòÏóê Îñ† ÏûàÎäî Ï¥àÏäπÎã¨ Í∞ôÏùÄ ÏòÅÌôî                   1
5                      ÏÇ¨ÎûëÏùÑ Ìï¥Î≥∏ÏÇ¨ÎûåÏù¥ÎùºÎ©¥ Ï≤òÏùåÎ∂ÄÌÑ∞ ÎÅùÍπåÏßÄ ÏõÉÏùÑÏàò ÏûàÎäîÏòÅÌôî                   1
6                                   ÏôÑÏ†Ñ Í∞êÎèôÏûÖÎãàÎã§ Îã§ÏãúÎ¥êÎèÑ Í∞êÎèô                   1
7                                 Í∞úÎì§Ïùò ÎÇòÏò§ÎÇòÏöî ÎÇòÏò§Î©¥ Î≥¥Í≥† Ïã∂Ïùå                   1
8  ÎÇ¥ ÎÇòÏù¥ÏôÄ Í∞ôÏùÄ ÏòÅÌôîÎ•º ÏßÄÍ∏à Î≥∏ ÎÇòÎäî Í∞êÎèôÏ†ÅÏù¥Îã§ÌïòÏßÄÎßå ÌõóÎÇ† Îã§ÏãúÎ≥¥Î©¥ÎåÄÏÇ¨ÌïòÎÇòÌïòÎÇò Í∑∏ ...                   1
9               ÏòÅÌôîÏùò Ïò§ÌéòÎùºÌôîÎùºÍ≥† Ìï¥ÏïºÌï† ÏûëÌíà Í∑πÎã®Ï†Å ÌèâÍ∞àÎ¶ºÏùÄ Ïñ¥Ï©î Ïàò ÏóÜÎäî ÎìØ                   0

In [53]:
a=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

In [54]:
selected_columns7['y_true']=pd.DataFrame(a)
selected_columns7 = selected_columns7.rename(columns={"category_prediction": "y_pred"})
selected_columns7.to_csv('/content/nsmc2_0.8_original_prediction.csv', encoding='utf-8-sig')

In [55]:
from sklearn.metrics import f1_score
import pandas as pd
df=pd.read_csv('/content/nsmc2_0.8_original_prediction.csv', encoding='utf-8-sig')

from sklearn.metrics import classification_report, f1_score

# y_trueÏôÄ y_predÎ•º Ï∂îÏ∂ú
y_true = df['y_true']
y_pred = df['y_pred']

# classification_reportÎ•º ÏÇ¨Ïö©ÌïòÏó¨ Í∞Å ÌÅ¥ÎûòÏä§Î≥Ñ F1-score Ï∂úÎ†•
report = classification_report(y_true, y_pred, digits=4)
print(report)

              precision    recall  f1-score   support

           0     0.8362    0.8220    0.8290      1000
           1     0.8250    0.8390    0.8319      1000

    accuracy                         0.8305      2000
   macro avg     0.8306    0.8305    0.8305      2000
weighted avg     0.8306    0.8305    0.8305      2000



# 50% Îç∞Ïù¥ÌÑ∞ÏÖã

In [56]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/·Ñå·Ö©·ÜØ·Ñã·Ö•·Ü∏·ÑÇ·Ö©·Ü´·ÑÜ·ÖÆ·Ü´/NSMC2_train_·Ñá·ÖÆ·ÜØ·ÑÄ·Ö≤·Ü´·Ñí·Öß·Üº_0.5.csv', encoding='utf-8-sig')

In [57]:
df_test=pd.read_excel('/content/gdrive/MyDrive/·Ñå·Ö©·ÜØ·Ñã·Ö•·Ü∏·ÑÇ·Ö©·Ü´·ÑÜ·ÖÆ·Ü´/DATA/NSMC_test.xlsx')

In [58]:
data_list = []
for q, label in zip(df['document'], df['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

#from sklearn.model_selection import train_test_split
#dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0, shuffle=True)

train=pd.DataFrame(data_list)

data_list = []
for q, label in zip(df_test['document'], df_test['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

test=pd.DataFrame(data_list)

train.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)
test.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)

def clean_korean_documents(documents):
    # ÌÖçÏä§Ìä∏ Ï†ïÏ†ú (ÌäπÏàòÍ∏∞Ìò∏Îßå Ï†úÍ±∞)
    for i, document in tqdm(enumerate(documents), total=len(documents), desc="Cleaning Special Characters"):
        document = re.sub(r'[^\w\s„Ñ±-„Ö£Í∞Ä-Ìû£]', '', document)  # ÌäπÏàòÍ∏∞Ìò∏Îßå Ï†úÍ±∞, Ï†ïÍ∑ú ÌëúÌòÑÏãù
        documents[i] = document
    return documents

def remove_words_with_numbers(text):
    return ' '.join([word for word in text.split(',') if not re.search(r'\d', word)])

# ÌïúÍ∏ÄÏù¥ ÏïÑÎãå Î¨∏ÏûêÎ•º Ï†úÍ±∞ÌïòÎäî Ìï®Ïàò
def remove_non_korean(text):
    return re.sub(r'[^„Ñ±-„Ö£Í∞Ä-Ìû£\s]', '', text)


train['text'] = clean_korean_documents(train['text'])
train['text'] = train['text'].apply(lambda x: ','.join(x.split()))
train['text'] = train['text'].apply(remove_words_with_numbers)
train['text'] = train['text'].apply(remove_non_korean)

test['text'] = clean_korean_documents(test['text'])
test['text'] = test['text'].apply(lambda x: ','.join(x.split()))
test['text'] = test['text'].apply(remove_words_with_numbers)
test['text'] = test['text'].apply(remove_non_korean)

# 3. ÌïôÏäµ Îç∞Ïù¥ÌÑ∞ÏôÄ ÌÖåÏä§Ìä∏ Îç∞Ïù¥ÌÑ∞ÏóêÏÑú ÌïÑÏöîÌïú Ïó¥ Ï∂îÏ∂ú Î∞è ÎùºÎ≤® Ïù∏ÏΩîÎî©
X_train = train['text']
y_train = train['category']
X_test = test['text']
y_test=test['category']

# 4. ÎùºÎ≤® Ïù∏ÏΩîÎî©
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded=label_encoder.fit_transform(y_test)
num_labels = len(label_encoder.classes_)

# 5. KoBERT Î™®Îç∏ÏùÑ ÏúÑÌïú ÌÜ†ÌÅ¨ÎÇòÏù¥Ï†Ä Î∞è Îç∞Ïù¥ÌÑ∞ÏÖã ÏÉùÏÑ±
model_name = 'monologg/kobert'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# ÌÖçÏä§Ìä∏Î•º ÌÜ†ÌÅ∞ÌôîÌïòÎäî Ìï®Ïàò Ï†ïÏùò
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=256)

# Îç∞Ïù¥ÌÑ∞ÏÖã ÏÉùÏÑ± Î∞è ÌÜ†ÌÅ¨ÎÇòÏù¥Ï¶à
train_df = pd.DataFrame({'text': X_train, 'label': y_train_encoded})
test_df = pd.DataFrame({'text': X_test, 'label': y_test_encoded})
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# PyTorch ÌÖêÏÑú ÌòïÏãùÏúºÎ°ú Î≥ÄÌôò
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# 5. KoBERT Î™®Îç∏ ÏÑ§Ï†ï Î∞è ÌïôÏäµ ÌååÎùºÎØ∏ÌÑ∞ ÏµúÏ†ÅÌôî
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device)


Cleaning Special Characters: 100%|‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà| 2000/2000 [00:00<00:00, 12884.32it/s]


Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [59]:
#CPU Ï∏°Ï†ï

import psutil
import time
from transformers import Trainer, TrainingArguments

# CPU ÏÇ¨Ïö©Îüâ Î∞è ÏÜçÎèÑ Ï∏°Ï†ï Ìï®Ïàò
def measure_cpu_usage_and_time(train_function):
    start_time = time.time()  # ÏãúÏûë ÏãúÍ∞Ñ Í∏∞Î°ù
    start_cpu = psutil.cpu_percent(interval=None)  # ÏãúÏûë CPU ÏÇ¨Ïö©Îüâ

    train_function()  # ÌïôÏäµ Ìï®Ïàò Ìò∏Ï∂ú

    end_time = time.time()  # Ï¢ÖÎ£å ÏãúÍ∞Ñ Í∏∞Î°ù
    end_cpu = psutil.cpu_percent(interval=None)  # Ï¢ÖÎ£å CPU ÏÇ¨Ïö©Îüâ

    elapsed_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    print(f"Elapsed Time: {elapsed_time:.2f} seconds")
    print(f"Average CPU Usage: {avg_cpu_usage:.2f}%")

In [60]:
import optuna
from transformers import Trainer, TrainingArguments
from sklearn.metrics import f1_score
import numpy as np

# `compute_metrics` Ìï®Ïàò
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='macro')  # F1 Ï†êÏàò Í≥ÑÏÇ∞
    return {'eval_f1': f1}  # 'eval_f1' Î∞òÌôò

# OptunaÎ•º ÌôúÏö©Ìïú ÌïòÏù¥ÌçºÌååÎùºÎØ∏ÌÑ∞ ÌäúÎãù
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

training_args = TrainingArguments(
    output_dir='/content',
    num_train_epochs=10,  # ÏóêÌè¨ÌÅ¨ Ïàò Í≥†Ï†ï
    per_device_train_batch_size=64,
    learning_rate=5.0e-05,  # ÏµúÏ†ÅÌôîÎêú ÌïôÏäµÎ•†
    weight_decay=0.01,  # ÏµúÏ†ÅÌôîÎêú weight decay
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='/content',
    logging_steps=10,
    fp16=True,
    optim="adamw_torch",  # AdamW fused Ï†ÅÏö© (PyTorch 2.0 Ïù¥ÏÉÅ)
    disable_tqdm=False,
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # Í∞ÄÏû• Ï¢ãÏùÄ Î™®Îç∏ÏùÑ accuracy Í∏∞Ï§ÄÏúºÎ°ú ÏÑ†ÌÉù
    greater_is_better=True,  # accuracyÍ∞Ä ÎÜíÏùÄ Î™®Îç∏ÏùÑ Ï†ÄÏû•
    save_total_limit=2,  # Ï†ÄÏû•Ìï† Î™®Îç∏ Í∞úÏàò Ï†úÌïú
    report_to="none"
)

# ÏµúÏ†ÅÏùò ÌååÎùºÎØ∏ÌÑ∞Î°ú Trainer Íµ¨ÏÑ±
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

trainer7 = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,  # Ï†ÑÏ≤¥ ÌïôÏäµ Îç∞Ïù¥ÌÑ∞Î°ú ÌèâÍ∞Ä
    compute_metrics=compute_metrics
)

# CPU ÏÇ¨Ïö©Îüâ Î∞è ÏÜçÎèÑ Ï∏°Ï†ï
measure_cpu_usage_and_time(trainer7.train)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of ü§ó Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Elapsed Time: 850.62 seconds
Average CPU Usage: 17.55%


In [61]:
# Î™®Îç∏ Ï†ÄÏû•
joblib.dump(trainer7, '/content/gdrive/MyDrive/NSMC2_0.5_original.pkl')

print("Î™®Îç∏Ïù¥ Ï†ÄÏû•ÎêòÏóàÏäµÎãàÎã§.")

Î™®Îç∏Ïù¥ Ï†ÄÏû•ÎêòÏóàÏäµÎãàÎã§.


In [62]:
# 8. Ï†ÄÏû•Îêú Î™®Îç∏Î°ú ÌÖåÏä§Ìä∏ Îç∞Ïù¥ÌÑ∞ ÏòàÏ∏° Î∞è Í≤∞Í≥º Ï†ÄÏû•
# Ï†ÄÏû•Îêú Î™®Îç∏ Î∂àÎü¨Ïò§Í∏∞

loaded_model = joblib.load('/content/gdrive/MyDrive/NSMC2_0.5_original.pkl')

# Î∂àÎü¨Ïò® Î™®Îç∏ÏùÑ ÏÇ¨Ïö©ÌïòÏó¨ ÏòàÏ∏° ÏàòÌñâ
predictions7 = loaded_model.predict(test_dataset)

test_predictions7 = np.argmax(predictions7.predictions, axis=1)
decoded_test_predictions7 = label_encoder.inverse_transform(test_predictions7)

test["category_prediction"] = decoded_test_predictions7

selected_columns7 = test[['text', 'category_prediction']]

a=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

selected_columns7['y_true']=pd.DataFrame(a)
selected_columns7 = selected_columns7.rename(columns={"category_prediction": "y_pred"})
selected_columns7.to_csv('/content/nsmc2_0.5_original_prediction.csv', encoding='utf-8-sig')



In [63]:
from sklearn.metrics import f1_score
import pandas as pd

df=pd.read_csv('/content/nsmc2_0.5_original_prediction.csv', encoding='utf-8-sig')

from sklearn.metrics import classification_report, f1_score

# y_trueÏôÄ y_predÎ•º Ï∂îÏ∂ú
y_true = df['y_true']
y_pred = df['y_pred']

# classification_reportÎ•º ÏÇ¨Ïö©ÌïòÏó¨ Í∞Å ÌÅ¥ÎûòÏä§Î≥Ñ F1-score Ï∂úÎ†•
report = classification_report(y_true, y_pred, digits=4)
print(report)

              precision    recall  f1-score   support

           0     0.8753    0.7650    0.8164      1000
           1     0.7913    0.8910    0.8382      1000

    accuracy                         0.8280      2000
   macro avg     0.8333    0.8280    0.8273      2000
weighted avg     0.8333    0.8280    0.8273      2000



# 10% Îç∞Ïù¥ÌÑ∞ÏÖã

In [64]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/·Ñå·Ö©·ÜØ·Ñã·Ö•·Ü∏·ÑÇ·Ö©·Ü´·ÑÜ·ÖÆ·Ü´/NSMC2_train_·Ñá·ÖÆ·ÜØ·ÑÄ·Ö≤·Ü´·Ñí·Öß·Üº_0.1.csv', encoding='utf-8-sig')

df_test=pd.read_excel('/content/gdrive/MyDrive/·Ñå·Ö©·ÜØ·Ñã·Ö•·Ü∏·ÑÇ·Ö©·Ü´·ÑÜ·ÖÆ·Ü´/DATA/NSMC_test.xlsx')

In [65]:
data_list = []
for q, label in zip(df['document'], df['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

#from sklearn.model_selection import train_test_split
#dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0, shuffle=True)

train=pd.DataFrame(data_list)

data_list = []
for q, label in zip(df_test['document'], df_test['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

test=pd.DataFrame(data_list)

train.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)
test.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)

def clean_korean_documents(documents):
    # ÌÖçÏä§Ìä∏ Ï†ïÏ†ú (ÌäπÏàòÍ∏∞Ìò∏Îßå Ï†úÍ±∞)
    for i, document in tqdm(enumerate(documents), total=len(documents), desc="Cleaning Special Characters"):
        document = re.sub(r'[^\w\s„Ñ±-„Ö£Í∞Ä-Ìû£]', '', document)  # ÌäπÏàòÍ∏∞Ìò∏Îßå Ï†úÍ±∞, Ï†ïÍ∑ú ÌëúÌòÑÏãù
        documents[i] = document
    return documents

def remove_words_with_numbers(text):
    return ' '.join([word for word in text.split(',') if not re.search(r'\d', word)])

# ÌïúÍ∏ÄÏù¥ ÏïÑÎãå Î¨∏ÏûêÎ•º Ï†úÍ±∞ÌïòÎäî Ìï®Ïàò
def remove_non_korean(text):
    return re.sub(r'[^„Ñ±-„Ö£Í∞Ä-Ìû£\s]', '', text)


train['text'] = clean_korean_documents(train['text'])
train['text'] = train['text'].apply(lambda x: ','.join(x.split()))
train['text'] = train['text'].apply(remove_words_with_numbers)
train['text'] = train['text'].apply(remove_non_korean)

test['text'] = clean_korean_documents(test['text'])
test['text'] = test['text'].apply(lambda x: ','.join(x.split()))
test['text'] = test['text'].apply(remove_words_with_numbers)
test['text'] = test['text'].apply(remove_non_korean)

# 3. ÌïôÏäµ Îç∞Ïù¥ÌÑ∞ÏôÄ ÌÖåÏä§Ìä∏ Îç∞Ïù¥ÌÑ∞ÏóêÏÑú ÌïÑÏöîÌïú Ïó¥ Ï∂îÏ∂ú Î∞è ÎùºÎ≤® Ïù∏ÏΩîÎî©
X_train = train['text']
y_train = train['category']
X_test = test['text']
y_test=test['category']

# 4. ÎùºÎ≤® Ïù∏ÏΩîÎî©
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded=label_encoder.fit_transform(y_test)
num_labels = len(label_encoder.classes_)

# 5. KoBERT Î™®Îç∏ÏùÑ ÏúÑÌïú ÌÜ†ÌÅ¨ÎÇòÏù¥Ï†Ä Î∞è Îç∞Ïù¥ÌÑ∞ÏÖã ÏÉùÏÑ±
model_name = 'monologg/kobert'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# ÌÖçÏä§Ìä∏Î•º ÌÜ†ÌÅ∞ÌôîÌïòÎäî Ìï®Ïàò Ï†ïÏùò
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=256)

# Îç∞Ïù¥ÌÑ∞ÏÖã ÏÉùÏÑ± Î∞è ÌÜ†ÌÅ¨ÎÇòÏù¥Ï¶à
train_df = pd.DataFrame({'text': X_train, 'label': y_train_encoded})
test_df = pd.DataFrame({'text': X_test, 'label': y_test_encoded})
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# PyTorch ÌÖêÏÑú ÌòïÏãùÏúºÎ°ú Î≥ÄÌôò
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# 5. KoBERT Î™®Îç∏ ÏÑ§Ï†ï Î∞è ÌïôÏäµ ÌååÎùºÎØ∏ÌÑ∞ ÏµúÏ†ÅÌôî
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device)


Cleaning Special Characters: 100%|‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà| 2000/2000 [00:00<00:00, 12586.97it/s]


Map:   0%|          | 0/5500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [66]:
#CPU Ï∏°Ï†ï

import psutil
import time
from transformers import Trainer, TrainingArguments

# CPU ÏÇ¨Ïö©Îüâ Î∞è ÏÜçÎèÑ Ï∏°Ï†ï Ìï®Ïàò
def measure_cpu_usage_and_time(train_function):
    start_time = time.time()  # ÏãúÏûë ÏãúÍ∞Ñ Í∏∞Î°ù
    start_cpu = psutil.cpu_percent(interval=None)  # ÏãúÏûë CPU ÏÇ¨Ïö©Îüâ

    train_function()  # ÌïôÏäµ Ìï®Ïàò Ìò∏Ï∂ú

    end_time = time.time()  # Ï¢ÖÎ£å ÏãúÍ∞Ñ Í∏∞Î°ù
    end_cpu = psutil.cpu_percent(interval=None)  # Ï¢ÖÎ£å CPU ÏÇ¨Ïö©Îüâ

    elapsed_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    print(f"Elapsed Time: {elapsed_time:.2f} seconds")
    print(f"Average CPU Usage: {avg_cpu_usage:.2f}%")

In [67]:
import optuna
from transformers import Trainer, TrainingArguments
from sklearn.metrics import f1_score
import numpy as np

# `compute_metrics` Ìï®Ïàò
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='macro')  # F1 Ï†êÏàò Í≥ÑÏÇ∞
    return {'eval_f1': f1}  # 'eval_f1' Î∞òÌôò

# OptunaÎ•º ÌôúÏö©Ìïú ÌïòÏù¥ÌçºÌååÎùºÎØ∏ÌÑ∞ ÌäúÎãù
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

training_args = TrainingArguments(
    output_dir='/content',
    num_train_epochs=10,  # ÏóêÌè¨ÌÅ¨ Ïàò Í≥†Ï†ï
    per_device_train_batch_size=64,
    learning_rate=5.0e-05,  # ÏµúÏ†ÅÌôîÎêú ÌïôÏäµÎ•†
    weight_decay=0.01,  # ÏµúÏ†ÅÌôîÎêú weight decay
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='/content',
    logging_steps=10,
    fp16=True,
    optim="adamw_torch",  # AdamW fused Ï†ÅÏö© (PyTorch 2.0 Ïù¥ÏÉÅ)
    disable_tqdm=False,
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # Í∞ÄÏû• Ï¢ãÏùÄ Î™®Îç∏ÏùÑ accuracy Í∏∞Ï§ÄÏúºÎ°ú ÏÑ†ÌÉù
    greater_is_better=True,  # accuracyÍ∞Ä ÎÜíÏùÄ Î™®Îç∏ÏùÑ Ï†ÄÏû•
    save_total_limit=2,  # Ï†ÄÏû•Ìï† Î™®Îç∏ Í∞úÏàò Ï†úÌïú
    report_to="none"
)

# ÏµúÏ†ÅÏùò ÌååÎùºÎØ∏ÌÑ∞Î°ú Trainer Íµ¨ÏÑ±
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

trainer7 = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,  # Ï†ÑÏ≤¥ ÌïôÏäµ Îç∞Ïù¥ÌÑ∞Î°ú ÌèâÍ∞Ä
    compute_metrics=compute_metrics
)

# CPU ÏÇ¨Ïö©Îüâ Î∞è ÏÜçÎèÑ Ï∏°Ï†ï
measure_cpu_usage_and_time(trainer7.train)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of ü§ó Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Elapsed Time: 647.70 seconds
Average CPU Usage: 17.95%


In [68]:
# Î™®Îç∏ Ï†ÄÏû•
joblib.dump(trainer7, '/content/gdrive/MyDrive/NSMC2_0.1_original.pkl')

print("Î™®Îç∏Ïù¥ Ï†ÄÏû•ÎêòÏóàÏäµÎãàÎã§.")

Î™®Îç∏Ïù¥ Ï†ÄÏû•ÎêòÏóàÏäµÎãàÎã§.


In [69]:
# 8. Ï†ÄÏû•Îêú Î™®Îç∏Î°ú ÌÖåÏä§Ìä∏ Îç∞Ïù¥ÌÑ∞ ÏòàÏ∏° Î∞è Í≤∞Í≥º Ï†ÄÏû•
# Ï†ÄÏû•Îêú Î™®Îç∏ Î∂àÎü¨Ïò§Í∏∞

loaded_model = joblib.load('/content/gdrive/MyDrive/NSMC2_0.1_original.pkl')

# Î∂àÎü¨Ïò® Î™®Îç∏ÏùÑ ÏÇ¨Ïö©ÌïòÏó¨ ÏòàÏ∏° ÏàòÌñâ
predictions7 = loaded_model.predict(test_dataset)

test_predictions7 = np.argmax(predictions7.predictions, axis=1)
decoded_test_predictions7 = label_encoder.inverse_transform(test_predictions7)

test["category_prediction"] = decoded_test_predictions7

selected_columns7 = test[['text', 'category_prediction']]

a=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

selected_columns7['y_true']=pd.DataFrame(a)
selected_columns7 = selected_columns7.rename(columns={"category_prediction": "y_pred"})
selected_columns7.to_csv('/content/nsmc2_0.1_original_prediction.csv', encoding='utf-8-sig')



In [70]:
from sklearn.metrics import f1_score
import pandas as pd

df=pd.read_csv('/content/nsmc2_0.1_original_prediction.csv', encoding='utf-8-sig')

from sklearn.metrics import classification_report, f1_score

# y_trueÏôÄ y_predÎ•º Ï∂îÏ∂ú
y_true = df['y_true']
y_pred = df['y_pred']

# classification_reportÎ•º ÏÇ¨Ïö©ÌïòÏó¨ Í∞Å ÌÅ¥ÎûòÏä§Î≥Ñ F1-score Ï∂úÎ†•
report = classification_report(y_true, y_pred, digits=4)
print(report)

              precision    recall  f1-score   support

           0     0.9255    0.4350    0.5918      1000
           1     0.6307    0.9650    0.7628      1000

    accuracy                         0.7000      2000
   macro avg     0.7781    0.7000    0.6773      2000
weighted avg     0.7781    0.7000    0.6773      2000



# 2% Îç∞Ïù¥ÌÑ∞ÏÖã

In [71]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/·Ñå·Ö©·ÜØ·Ñã·Ö•·Ü∏·ÑÇ·Ö©·Ü´·ÑÜ·ÖÆ·Ü´/NSMC2_train_·Ñá·ÖÆ·ÜØ·ÑÄ·Ö≤·Ü´·Ñí·Öß·Üº_0.02.csv', encoding='utf-8-sig')

df_test=pd.read_excel('/content/gdrive/MyDrive/·Ñå·Ö©·ÜØ·Ñã·Ö•·Ü∏·ÑÇ·Ö©·Ü´·ÑÜ·ÖÆ·Ü´/DATA/NSMC_test.xlsx')

In [72]:
data_list = []
for q, label in zip(df['document'], df['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

#from sklearn.model_selection import train_test_split
#dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0, shuffle=True)

train=pd.DataFrame(data_list)

data_list = []
for q, label in zip(df_test['document'], df_test['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

test=pd.DataFrame(data_list)

train.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)
test.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)

def clean_korean_documents(documents):
    # ÌÖçÏä§Ìä∏ Ï†ïÏ†ú (ÌäπÏàòÍ∏∞Ìò∏Îßå Ï†úÍ±∞)
    for i, document in tqdm(enumerate(documents), total=len(documents), desc="Cleaning Special Characters"):
        document = re.sub(r'[^\w\s„Ñ±-„Ö£Í∞Ä-Ìû£]', '', document)  # ÌäπÏàòÍ∏∞Ìò∏Îßå Ï†úÍ±∞, Ï†ïÍ∑ú ÌëúÌòÑÏãù
        documents[i] = document
    return documents

def remove_words_with_numbers(text):
    return ' '.join([word for word in text.split(',') if not re.search(r'\d', word)])

# ÌïúÍ∏ÄÏù¥ ÏïÑÎãå Î¨∏ÏûêÎ•º Ï†úÍ±∞ÌïòÎäî Ìï®Ïàò
def remove_non_korean(text):
    return re.sub(r'[^„Ñ±-„Ö£Í∞Ä-Ìû£\s]', '', text)


train['text'] = clean_korean_documents(train['text'])
train['text'] = train['text'].apply(lambda x: ','.join(x.split()))
train['text'] = train['text'].apply(remove_words_with_numbers)
train['text'] = train['text'].apply(remove_non_korean)

test['text'] = clean_korean_documents(test['text'])
test['text'] = test['text'].apply(lambda x: ','.join(x.split()))
test['text'] = test['text'].apply(remove_words_with_numbers)
test['text'] = test['text'].apply(remove_non_korean)

# 3. ÌïôÏäµ Îç∞Ïù¥ÌÑ∞ÏôÄ ÌÖåÏä§Ìä∏ Îç∞Ïù¥ÌÑ∞ÏóêÏÑú ÌïÑÏöîÌïú Ïó¥ Ï∂îÏ∂ú Î∞è ÎùºÎ≤® Ïù∏ÏΩîÎî©
X_train = train['text']
y_train = train['category']
X_test = test['text']
y_test=test['category']

# 4. ÎùºÎ≤® Ïù∏ÏΩîÎî©
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded=label_encoder.fit_transform(y_test)
num_labels = len(label_encoder.classes_)

# 5. KoBERT Î™®Îç∏ÏùÑ ÏúÑÌïú ÌÜ†ÌÅ¨ÎÇòÏù¥Ï†Ä Î∞è Îç∞Ïù¥ÌÑ∞ÏÖã ÏÉùÏÑ±
model_name = 'monologg/kobert'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# ÌÖçÏä§Ìä∏Î•º ÌÜ†ÌÅ∞ÌôîÌïòÎäî Ìï®Ïàò Ï†ïÏùò
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=256)

# Îç∞Ïù¥ÌÑ∞ÏÖã ÏÉùÏÑ± Î∞è ÌÜ†ÌÅ¨ÎÇòÏù¥Ï¶à
train_df = pd.DataFrame({'text': X_train, 'label': y_train_encoded})
test_df = pd.DataFrame({'text': X_test, 'label': y_test_encoded})
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# PyTorch ÌÖêÏÑú ÌòïÏãùÏúºÎ°ú Î≥ÄÌôò
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# 5. KoBERT Î™®Îç∏ ÏÑ§Ï†ï Î∞è ÌïôÏäµ ÌååÎùºÎØ∏ÌÑ∞ ÏµúÏ†ÅÌôî
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device)

#CPU Ï∏°Ï†ï

import psutil
import time
from transformers import Trainer, TrainingArguments

# CPU ÏÇ¨Ïö©Îüâ Î∞è ÏÜçÎèÑ Ï∏°Ï†ï Ìï®Ïàò
def measure_cpu_usage_and_time(train_function):
    start_time = time.time()  # ÏãúÏûë ÏãúÍ∞Ñ Í∏∞Î°ù
    start_cpu = psutil.cpu_percent(interval=None)  # ÏãúÏûë CPU ÏÇ¨Ïö©Îüâ

    train_function()  # ÌïôÏäµ Ìï®Ïàò Ìò∏Ï∂ú

    end_time = time.time()  # Ï¢ÖÎ£å ÏãúÍ∞Ñ Í∏∞Î°ù
    end_cpu = psutil.cpu_percent(interval=None)  # Ï¢ÖÎ£å CPU ÏÇ¨Ïö©Îüâ

    elapsed_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    print(f"Elapsed Time: {elapsed_time:.2f} seconds")
    print(f"Average CPU Usage: {avg_cpu_usage:.2f}%")

Cleaning Special Characters: 100%|‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà| 2000/2000 [00:00<00:00, 13068.44it/s]


Map:   0%|          | 0/5100 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [73]:
import optuna
from transformers import Trainer, TrainingArguments
from sklearn.metrics import f1_score
import numpy as np

# `compute_metrics` Ìï®Ïàò
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='macro')  # F1 Ï†êÏàò Í≥ÑÏÇ∞
    return {'eval_f1': f1}  # 'eval_f1' Î∞òÌôò

# OptunaÎ•º ÌôúÏö©Ìïú ÌïòÏù¥ÌçºÌååÎùºÎØ∏ÌÑ∞ ÌäúÎãù
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

training_args = TrainingArguments(
    output_dir='/content',
    num_train_epochs=10,  # ÏóêÌè¨ÌÅ¨ Ïàò Í≥†Ï†ï
    per_device_train_batch_size=64,
    learning_rate=5.0e-05,  # ÏµúÏ†ÅÌôîÎêú ÌïôÏäµÎ•†
    weight_decay=0.01,  # ÏµúÏ†ÅÌôîÎêú weight decay
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='/content',
    logging_steps=10,
    fp16=True,
    optim="adamw_torch",  # AdamW fused Ï†ÅÏö© (PyTorch 2.0 Ïù¥ÏÉÅ)
    disable_tqdm=False,
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # Í∞ÄÏû• Ï¢ãÏùÄ Î™®Îç∏ÏùÑ accuracy Í∏∞Ï§ÄÏúºÎ°ú ÏÑ†ÌÉù
    greater_is_better=True,  # accuracyÍ∞Ä ÎÜíÏùÄ Î™®Îç∏ÏùÑ Ï†ÄÏû•
    save_total_limit=2,  # Ï†ÄÏû•Ìï† Î™®Îç∏ Í∞úÏàò Ï†úÌïú
    report_to="none"
)

# ÏµúÏ†ÅÏùò ÌååÎùºÎØ∏ÌÑ∞Î°ú Trainer Íµ¨ÏÑ±
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

trainer7 = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,  # Ï†ÑÏ≤¥ ÌïôÏäµ Îç∞Ïù¥ÌÑ∞Î°ú ÌèâÍ∞Ä
    compute_metrics=compute_metrics
)

# CPU ÏÇ¨Ïö©Îüâ Î∞è ÏÜçÎèÑ Ï∏°Ï†ï
measure_cpu_usage_and_time(trainer7.train)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of ü§ó Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Elapsed Time: 607.23 seconds
Average CPU Usage: 19.10%


In [74]:
# Î™®Îç∏ Ï†ÄÏû•
joblib.dump(trainer7, '/content/gdrive/MyDrive/NSMC2_0.02_original.pkl')

print("Î™®Îç∏Ïù¥ Ï†ÄÏû•ÎêòÏóàÏäµÎãàÎã§.")

Î™®Îç∏Ïù¥ Ï†ÄÏû•ÎêòÏóàÏäµÎãàÎã§.


In [75]:
# 8. Ï†ÄÏû•Îêú Î™®Îç∏Î°ú ÌÖåÏä§Ìä∏ Îç∞Ïù¥ÌÑ∞ ÏòàÏ∏° Î∞è Í≤∞Í≥º Ï†ÄÏû•
# Ï†ÄÏû•Îêú Î™®Îç∏ Î∂àÎü¨Ïò§Í∏∞

loaded_model = joblib.load('/content/gdrive/MyDrive/NSMC2_0.02_original.pkl')

# Î∂àÎü¨Ïò® Î™®Îç∏ÏùÑ ÏÇ¨Ïö©ÌïòÏó¨ ÏòàÏ∏° ÏàòÌñâ
predictions7 = loaded_model.predict(test_dataset)

test_predictions7 = np.argmax(predictions7.predictions, axis=1)
decoded_test_predictions7 = label_encoder.inverse_transform(test_predictions7)

test["category_prediction"] = decoded_test_predictions7

selected_columns7 = test[['text', 'category_prediction']]

a=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

selected_columns7['y_true']=pd.DataFrame(a)
selected_columns7 = selected_columns7.rename(columns={"category_prediction": "y_pred"})
selected_columns7.to_csv('/content/nsmc2_0.02_original_prediction.csv', encoding='utf-8-sig')



In [76]:
from sklearn.metrics import f1_score
import pandas as pd

df=pd.read_csv('/content/nsmc2_0.02_original_prediction.csv', encoding='utf-8-sig')

from sklearn.metrics import classification_report, f1_score

# y_trueÏôÄ y_predÎ•º Ï∂îÏ∂ú
y_true = df['y_true']
y_pred = df['y_pred']

# classification_reportÎ•º ÏÇ¨Ïö©ÌïòÏó¨ Í∞Å ÌÅ¥ÎûòÏä§Î≥Ñ F1-score Ï∂úÎ†•
report = classification_report(y_true, y_pred, digits=4)
print(report)

              precision    recall  f1-score   support

           0     0.8866    0.0860    0.1568      1000
           1     0.5197    0.9890    0.6814      1000

    accuracy                         0.5375      2000
   macro avg     0.7032    0.5375    0.4191      2000
weighted avg     0.7032    0.5375    0.4191      2000

